In [226]:
import os
import json
import pandas as pd
import re

from openai import OpenAI

## 데이터 전처리

In [227]:
# 엑셀 파일 경로
file_path = '../data/arts_all.xlsx'
df = pd.read_excel(file_path)
df = df[df['작품이미지'] == 1].reset_index(drop=True)
df["이미지"] = df['번호'].apply(lambda x: str(int(x))+".jpg")
# df = df[0:50]
df.to_csv("../data/arts_all.csv", index=False)

In [228]:
# 엑셀 파일 경로
file_path = '../data/exhibitions.xlsx'
df = pd.read_excel(file_path)
df = df[df['이미지'] == 1].reset_index(drop=True)
df["이미지"] = df['번호'].apply(lambda x: str(x)+".jpg")
# df = df[0:50]
df.to_csv("../data/exhibitions.csv", index=False)

In [207]:
df = pd.read_csv("../data/arts_all.csv")
ehb_df = pd.read_csv("../data/exhibitions.csv")

In [194]:
df

,번호,작가,작품명,제작연도,재료,규격,부문,관리번호,작품 설명,작품이미지,이미지
0,1.0,이중섭,포도 따는 남자,194,"종이에 펜, 수채 물감",14 x 9,드로잉,10090.0,"대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에...",1.0,1.jpg
1,2.0,김기창,진돗개,1975,"종이에 먹, 색",66.3 x 95.6,회화1,9172.0,"운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장...",1.0,2.jpg
2,3.0,김기창,청산정경,1979,"비단에 먹, 색",68.2 x 136.6,회화1,9177.0,"운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장...",1.0,3.jpg
3,4.0,이응노,구상,1974,"캔버스에 천, 콜라주",204 x 126,회화1,9909.0,"고암(顧庵) 이응노(李應魯, 1904-1989)는 1922년에 동양화가인 김규진에게...",1.0,4.jpg
4,5.0,이응노,사자,1976,"종이에 먹, 색",42.7 x 59.5,회화1,9927.0,"고암(顧菴) 이응노(李應魯, 1904-1989)는 17세 무렵 서화가 송태회(宋泰會...",1.0,5.jpg
...,...,...,...,...,...,...,...,...,...,...,...
125,131.0,파블로 피카소 Pablo PICASSO,염소 머리,1952,"백토, 화장토, 파티나 장식, 부분 유약 시유",40×40×4,공예,10454.0,"파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Mál...",1.0,131.jpg
126,132.0,파블로 피카소 Pablo PICASSO,투우사(반데리예로),1959,"적토, 화장토 장식",40×40×1,공예,10411.0,"파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Mál...",1.0,132.jpg
127,134.0,파블로 피카소 Pablo PICASSO,어린 올빼미,1952,"백토, 산화물, 백색 에나멜 장식, 나이프 각인",24×9×10,공예,10397.0,"파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Mál...",1.0,134.jpg
128,135.0,파블로 피카소 Pablo PICASSO,작은 태양,1968 - 1969,"적토, 화장토, 산화물, 파티나 장식, 부분 유약 시유",9.5×16×2.5,NaN,10434.0,"파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Mál...",1.0,135.jpg


In [195]:
ehb_df

,번호,전시,작품리스트,이미지
0,1,MMCA 기증작품전: 1960-70년대 구상회화,"51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62...",1.jpg
1,2,MMCA 이건희컬렉션 특별전: 한국미술명작,"74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85...",2.jpg
2,3,"MMCA 이건희컬렉션 특별전: 모네와 피카소, 파리의 아름다운 순간들","97, 98, 99, 100, 101, 102, 103, 104, 105, 106,...",3.jpg
3,4,가변하는 소장품,"118, 119, 120, 121, 122, 123, 124, 125, 126, 127",4.jpg
4,5,국립현대미술관 소장품전 피카소 도예,"112, 129, 130, 105, 132, 116, 134, 135, 136",5.jpg


## Solar API
- 작가의 설명과 작품의 설명을 분리합니다.

In [13]:
prompt = """
## Role: 미술 작품 설명 추출기

## Instruction
- 미술 작품에 대한 설명이 주어지면 작가에 대한 설명과 작품에 대한 설명을 분리한다.
- 가능한 많은 내용을 담아 요약한다.
- 아래 Json 포맷에 맞게 답변을 생성한다.
- 한국어로 생성한다.

## Output format
{"art_description": "$art_description", "author_description":"$author_description"}
"""

for i in range(len(df)):
    row_data = df.loc[i]
    content = df['작품 설명'][i]

    client = OpenAI(
        api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
    )
    chat_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": content},
        ],
    )

    try:
        output_dict = json.loads(chat_result.choices[0].message.content)
        df.loc[i, 'art_description'] = output_dict['art_description']
        df.loc[i, 'author_description'] = output_dict['author_description']
    except:
        df.loc[i, 'art_description'] = "Error"
        df.loc[i, 'author_description'] = "Error"
    

### 데이터 확인

In [68]:
j_df = df.to_json(orient='records', force_ascii=False)
j_df = json.loads(j_df)

idx = 130
for data in j_df[idx:idx+10]:
    print(data['번호'])
    print(data['작가'])
    print(data['작품명'])
    # print("# 작품설명: ", data['작품 설명'])
    print("# art_description: ", data['art_description'])
    print("# author_description: ", data['author_description'])
    print()



131
파블로 피카소 Pablo PICASSO
투우와 사람들
# art_description:  파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Málaga)에서 태어나 미술 교사인 아버지에게 일곱살까지 미술 교육을 받고, 바르셀로나와 마드리드 미술학교를 거치며 일찌감치 아카데믹한 화풍으로 뛰어난 기량을 보였다. 1904년 파리로 건너가 영구 정착하였으며, 무쟁(Mougins)에서 사망하기까지 프랑스를 주 무대로 활동하면서 전 생애에 걸친 왕성한 창작활동을 펼쳤다. 가난과 우울, 내면의 고통을 청색의 단색조로 형상화하던 청색 시대(1901–1904)와 감상적인 주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Cézanne, 1839-1906)과 원시미술의 영향을 받은 <아비뇽의 여인들(Les Demoiselles d'Avignon)>(1906–1907)을 통해 혁명적 화풍의 시작을 알렸다. 조르주 브라크(Georges Braque, 1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새로운 조형 언어를 선보였으며 이후 신고전주 의와 초현실주의적 요소를 활용해 방대한 양의 회화, 조각, 드로잉, 판화, 공예 작품을 제작하였다. 또한 1937년에 제작한 <게르니카(Guernica)>(1937)를 통해 역사화를 부활시켰다. 피카소는 다양한 재료의 효과와 재질감의 대비로부터 나오는 표현성을 보편화시키고, 콜라주나 아상블라주와 같은 당대 혁신적인 조형 작업을 선보이며 장르 불문의 새로운 시각예술 언어를 제시하였다. 그는 인물 초상, 사생활의 소산, 생의 희비극, 정치적 메시지, 대담하고 우아한 에로티시즘 등 다양한 주제를 망라했다.

<투우와 사람들(Taureau et Tauromachies)>은 피카소의 초기 도예 작품 중 하나로, 접시의 안쪽 움푹한 부분을 경기장으로 활용해 투우의 격정적인 장면을 담아냈다. 투우는 피카소의 스페인적 정체성을 드러내는 모티프 중 하나였다. 푸른색의 투우사

In [70]:
j_df[131]

{'번호': 132,
 '작가': '파블로 피카소 Pablo PICASSO',
 '작품명': '투우사(반데리예로)',
 '제작연도': '1959',
 '재료': '적토, 화장토 장식',
 '규격': '40×40×1',
 '부문': '공예',
 '관리번호': 10411.0,
 '작품 설명': "파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Málaga)에서 태어나 미술 교사인 아버지에게 일곱살까지 미술 교육을 받고, 바르셀로나와 마드리드 미술학교를 거치며 일찌감치 아카데믹한 화풍으로 뛰어난 기량을 보였다. 1904년 파리로 건너가 영구 정착하였으며, 무쟁(Mougins)에서 사망하기까지 프랑스를 주 무대로 활동하면서 전 생애에 걸친 왕성한 창작활동을 펼쳤다. 가난과 우울, 내면의 고통을 청색의 단색조로 형상화하던 청색 시대(1901–1904)와 감상적인 주제와 색조의 장밋빛 시대(1904–1906)를 지나, 폴 세잔(Paul Cézanne, 1839-1906)과 원시미술의 영향을 받은 〈아비뇽의 여인들(Les Demoiselles d'Avignon)〉(1906–1907)을 통해 혁명적 화풍의 시작을 알렸다. 조르주 브라크(Georges Braque, 1882-1963)와 공동 창시한 큐비즘(1909–1919)을 통해 20세기의 새로운 조형 언어를 선보였으며 이후 신고전주 의와 초현실주의적 요소를 활용해 방대한 양의 회화, 조각, 드로잉, 판화, 공예 작품을 제작하였다. 또한 1937년에 제작한 〈게르니카(Guernica)〉를 통해 역사화를 부활시켰다. 피카소는 다양한 재료의 효과와 재질감의 대비로부터 나오는 표현성을 보편화시키고, 콜라주나 아상블라주와 같은 당대 혁신적인 조형 작업을 선보이며 장르 불문의 새로운 시각예술 언어를 제시하였다. 그는 인물 초상, 사생활의 소산, 생의 희비극, 정치적 메시지, 대담하고 우아한 에로티시즘 등 다양한 주제를 망라했다. 〈투우사(반데리예로)〉는 피카소가 적토로 투우 장면을 표현한 ‘접시’ 연작 

## Solar API 2차
- 품질이 낮거나 에러가 발생한 데이터 다시 호출

In [5]:
retry_num = [8, 9, 16, 17, 20, 29, 33, 39, 41, 43, 49, 58, 59, 62, 67, 69, 87, 88, 89, 93, 94, 97, 102, 105, 106, 109, 114, 119, 120, 128, 129, 131, 132]
retry_idx = [i-1 for i in retry_num]

In [82]:
prompt = """
## Role: 미술 작품 설명 추출기

## Instruction
- 미술 작품에 대한 설명이 주어지면 작가에 대한 설명과 작품에 대한 설명을 분리한다.
- 가능한 많은 내용을 담아 요약한다.
- 아래 Json 포맷에 맞게 답변을 생성한다.
- 한국어로 생성한다.

## Output format
{"art_description": "$art_description", "author_description":"$author_description"}
"""

for i in retry_idx:
    row_data = df.loc[i]
    content = df['작품 설명'][i]

    client = OpenAI(
        api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
    )
    chat_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": content},
        ],
    )

    try:
        output_dict = json.loads(chat_result.choices[0].message.content)
        df.loc[i, 'art_description'] = output_dict['art_description']
        df.loc[i, 'author_description'] = output_dict['author_description']
    except:
        df.loc[i, 'art_description'] = "Error"
        df.loc[i, 'author_description'] = "Error"
    

### 데이터 확인

In [9]:
j_df = df.iloc[retry_idx].to_json(orient='records', force_ascii=False)
j_df = json.loads(j_df)

idx = 0
for i, data in enumerate(j_df[idx:idx+100]):
    print(i)
    print(data['번호'])
    print(data['작가'])
    print(data['작품명'])
    # print("# 작품설명: ", data['작품 설명'])
    print("# art_description: ", data['art_description'])
    print("# author_description: ", data['author_description'])
    print()



0
8
이용우
삼고초려
# art_description:  <삼고초려>는 유비가 제갈량을 초빙하기 위해 초옥을 세 번 찾아간다는 내용의 ‘삼고초려(三顧草廬)’를 그린 고사인물화입니다. 이용우 작가는 1911년 서화미술회 1기생으로 입학하여 심전 안중식과 소림 조석진에게서 서화를 배웠으며, 1918년 서화협회의 최연소 정회원이 되었습니다. 그는 동양화가로서 입지를 다지고, 1920년대 후반에는 동아일보와 조선일보의 연작소설 삽화가로 활동했습니다. <삼고초려>는 근경에 큰 괴석과 자연물을 배치하였고, 중경에 인물들의 모습을, 멀리 원경에 이들이 타고 온 말과 자연풍경을 묘사했습니다. 작품 상단에는 효산 이광렬의 화제를 확인할 수 있습니다.
# author_description:  묵로 이용우는 1902년부터 1952년까지 활동한 한국의 동양화가입니다. 그는 9살에 서화미술회에 입학하여 심전 안중식과 소림 조석진에게서 서화를 배웠으며, 1918년 서화협회의 최연소 정회원이 되었습니다. 그는 고사인물도와 풍속화를 많이 그렸으며, 특유의 섬세한 필치와 화려하고 맑은 색감이 특징입니다. 해방 이후에는 《제1회 대한민국미술전람회》의 동양화부 심사위원으로 위촉되었으며, 1950년 한국전쟁 당시 전주로 피난하여 작품 활동을 했습니다.

1
9
김상우
마릴린
# art_description:  <마릴린>은 마릴린 먼로의 초상을 극사실주의적으로 묘사한 작품입니다. 작가는 인물의 형태를 왜곡하여 시점에 따라 부피가 다르게 보이게 함으로써 현대미의 기준에 대한 고정관념을 파기하려고 했습니다. 또한, 스티브 잡스의 전기를 읽거나 스마트폰으로 음악을 듣는 등 현대적인 모습의 먼로도 그렸습니다. 이는 현실에서는 연결될 수 없는 두 개의 시공간을 이어 붙여 먼로에게 새로운 삶의 기회를 주기 위한 시도였습니다.
# author_description:  김상우는 중앙대학교 미술대학 및 동 대학원을 졸업하고 러시아에서 미술을 공부했습니다. 그는 인물화를 실제 인체와 같은 크기로 제작하여 관람자에

In [8]:
j_df[4]

{'번호': 20,
 '작가': '한봉호 HAN Bongho',
 '작품명': '장미',
 '제작연도': '연도미상',
 '재료': '캔버스에 유화 물감',
 '규격': '72.7×60.6',
 '부문': '회화 II',
 '관리번호': 10478.0,
 '작품 설명': '한봉호(韓奉浩, 1921-2016)는 평안북도 의주 출생으로 일본으로 건너가 오랜 기간 일본 화단에서 활동하며, 1980년 일본 《제14회 현대미술전》에서 일본내각총리대신상, 1982년 《제15회 현대미술전》 최고상, 1991년 일본 이원전(二元展)에서는 일본문부대신상을 수상했다. 그는 국내보다는 유럽과 일본을 중심으로 활동했으며, 스페인 마드리드 등을 여행하고 제작한 플라멩코(flamenco) 춤을 추는 모습을 그린 ‘플라멩코’ 연작이 대표적이다. 그의 작품은 강렬한 필치, 표현주의적인 색채, 율동감이 두드러지는 특징을 보인다.\n\n <장미>는 탁상 위에 놓인 여러 기물과 화병에 꽂힌 장미를 묘사한 그림이다. 한봉호는 주로 순간을 포착하듯이 빠른 붓 터치와 강렬한 채색을 사용하여 이국적인 소재를 율동감 넘치게 표현한 회화로 대표되지만, 이 작품과 같이 정물이나 풍경 등을 그리기도 했다. 그러나 이 작품에서도 장미꽃, 화병, 유리잔, 탁상시계와 같은 정적인 소재에서 표현주의적인 기법의 묘사가 돋보인다. 물감의 채도를 조절하여 장미꽃과 이파리에 생동감을 부여하고, 배경에는 다채로운 색채로 명암을 표현해서 장미가 돋보이는 효과를 낸다. 작가는 빠른 붓 터치와 과감한 색채를 사용하여 장미에 대한 순간적인 인상을 작품에 담아냈다.\n\n',
 '작품이미지': 1,
 '이미지': '20.jpg',
 'art_description': '탁상 위에 놓인 여러 기물과 화병에 꽂힌 장미를 묘사한 정물화입니다. 장미꽃과 이파리에 생동감을 부여하고, 배경에는 다채로운 색채로 명암을 표현하여 장미가 돋보이는 효과를 냈습니다. 빠른 붓 터치와 과감한 색채를 사용하여 장미에 대한 순간적인 인상을 담아냈습니다.',


## Solar API 3차 호출
- 잘 처리되지 않는 데이터 개별로 처리

In [10]:
retry_num = [16, 20, 41, 88, 94, 97, 109, 114, 128]
retry_idx = [i-1 for i in retry_num]

In [116]:
i = 50
retry_idx

[15, 19, 40, 87, 93, 96, 108, 113, 127]

In [117]:
df.loc[i].to_dict()

{'번호': 51,
 '작가': '이병규 LEE Byeonggyu',
 '작품명': '고궁일우(古宮一隅)',
 '제작연도': '1961',
 '재료': '캔버스에 유화 물감',
 '규격': '99×130',
 '부문': '회화 II',
 '관리번호': 10830.0,
 '작품 설명': '오하(梧下) 이병규(李昞圭, 1901-1974)는 1920년 일본으로 건너가 도쿄미술학교(東京美術学校)에서 서양화를 공부했다. 귀국 후 조선미술전람회에는 출품하지 않고, 서화협회전 등의 전시에 참여하였다. 1934년 이종우, 이마동 등과 함께 민족적 의식을 표방한 화가 조직인 목일회(牧日會)를 창설하고, 이를 중심으로 활동했다. 그는 일본 유학 시기에 습득한 사실주의적 화풍을 바탕으로, 주변의 인물, 식물, 사찰, 자연 풍경을 주요 소재로 한 작품을 제작하였다. 1958년 목우회(木友會), 1968년 한국사실작가회를 창립하는 등 사실주의 계열의 미술 발전에 기여했으며, 대한민국미술전람회 초대작가, 추천작가 및 심사위원을 역임하면서 활발히 활동하였다. 1970년 대한민국 국민훈장 모란장, 1971년 대한민국 문화예술상을 받았다.',
 '작품이미지': 1,
 '이미지': '51.jpg',
 'art_description': '자연 풍경과 주변 인물, 식물, 사찰을 주요 소재로 한 작품',
 'author_description': '이병규는 일본의 사실주의적 화풍을 바탕으로 한 화가로, 목일회, 목우회, 한국사실작가회 등을 창립하여 사실주의 계열의 미술 발전에 기여했습니다.'}

In [122]:
prompt = """
## Role: 미술 작품 설명 추출기

## Instruction
- 미술 작품에 대한 설명이 주어지면 작가에 대한 설명과 작품에 대한 설명을 분리한다.
- 가능한 많은 내용을 담아 요약한다.
- 아래 Json 포맷에 맞게 답변을 생성한다.
- 한국어로 생성한다.

## Output format
{"art_description": "$art_description", "author_description":"$author_description"}
"""
row_data = df.loc[i]
content = df['작품 설명'][i]

client = OpenAI(
    api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
)
chat_result = client.chat.completions.create(
    model="solar-1-mini-chat",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": content},
    ],
)

try:
    output_dict = json.loads(chat_result.choices[0].message.content)
    df.loc[i, 'art_description'] = output_dict['art_description']
    df.loc[i, 'author_description'] = output_dict['author_description']
except:
    df.loc[i, 'art_description'] = "Error"
    df.loc[i, 'author_description'] = "Error"


In [123]:
df.loc[i].to_dict()

{'번호': 51,
 '작가': '이병규 LEE Byeonggyu',
 '작품명': '고궁일우(古宮一隅)',
 '제작연도': '1961',
 '재료': '캔버스에 유화 물감',
 '규격': '99×130',
 '부문': '회화 II',
 '관리번호': 10830.0,
 '작품 설명': '오하(梧下) 이병규(李昞圭, 1901-1974)는 1920년 일본으로 건너가 도쿄미술학교(東京美術学校)에서 서양화를 공부했다. 귀국 후 조선미술전람회에는 출품하지 않고, 서화협회전 등의 전시에 참여하였다. 1934년 이종우, 이마동 등과 함께 민족적 의식을 표방한 화가 조직인 목일회(牧日會)를 창설하고, 이를 중심으로 활동했다. 그는 일본 유학 시기에 습득한 사실주의적 화풍을 바탕으로, 주변의 인물, 식물, 사찰, 자연 풍경을 주요 소재로 한 작품을 제작하였다. 1958년 목우회(木友會), 1968년 한국사실작가회를 창립하는 등 사실주의 계열의 미술 발전에 기여했으며, 대한민국미술전람회 초대작가, 추천작가 및 심사위원을 역임하면서 활발히 활동하였다. 1970년 대한민국 국민훈장 모란장, 1971년 대한민국 문화예술상을 받았다.',
 '작품이미지': 1,
 '이미지': '51.jpg',
 'art_description': '이 작품은 주변의 인물, 식물, 사찰, 자연 풍경을 주요 소재로 한 작품입니다. 사실주의적 화풍을 바탕으로 그려졌으며, 주변의 일상적인 풍경을 세밀하게 표현하고 있습니다. 작가는 주변의 사물을 관찰하고 이를 그림으로 표현하여 현실적이고 생동감 넘치는 작품을 완성하였습니다. 이 작품은 작가의 섬세한 관찰력과 뛰어난 기술력을 통해 탄생한 사실주의적인 예술의 한 예입니다.',
 'author_description': '오하(梧下) 이병규(李昞圭, 1901-1974)는 한국의 서양화 화가로, 일본 도쿄미술학교에서 공부한 후 귀국하여 서화협회전 등에 참여하였습니다. 그는 민족적 의식을 표방한 화가 조직인 목일회를 창설하고, 사실주의적 화풍을 바탕으로 작품을 제작하였습니

In [129]:
j_df = df.to_json(orient='records', force_ascii=False)
j_df = json.loads(j_df)

idx = 0
for i, data in enumerate(j_df[idx:idx+200]):
    print(i)
    print(data['번호'])
    print(data['작가'])
    print(data['작품명'])
    # print("# 작품설명: ", data['작품 설명'])
    print("# art_description: ", data['art_description'])
    print("# author_description: ", data['author_description'])
    print()



0
1
이중섭
포도 따는 남자
# art_description:  이 작품은 포도나무 아래에서 포도를 따는 남자를 그린 그림입니다. 그림의 전경 부분은 펜으로 테두리를 그렸고, 포도가 매달려 있는 줄기와 배경의 인물은 수채 물감으로만 그려져 흐릿하게 보입니다. 포도알 하나가 인물의 손보다 크게 그려져 있어 상상적인 풍경을 연상시킵니다. 그림 속 포도를 따는 모습은 노동의 현장이라기보다는 풍요로운 동산에서 자유롭게 수확하는 느낌을 주며, 프랑스의 화가 폴 고갱의 타히티 시절 그림을 연상시킵니다.
# author_description:  이중섭은 평안북도 정주의 오산고등보통학교에서 임용련, 백남순 부부에게 서양화를 배웠으며, 이후 일본으로 건너가 데이코쿠미술학교와 분카학원에서 미술을 전공했습니다. 그는 자유미술가협회의 전시회에 지속적으로 출품하였으며, 제7회전에서는 태양상을 수상했습니다. 이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것에 영감을 받아 표현주의적인 감각으로 작품을 제작했습니다. 그의 작품에서는 동경 시절의 연애, 제주도 피란시절의 행복, 그리고 가족과 재회할 수 없다는 절망 등이 표현되어 있습니다.

1
2
김기창
진돗개
# art_description:  <진돗개>는 조화로운 색채와 적절한 먹의 농담 표현으로 개의 모습을 생동감 있게 전달한 작품입니다. 어미 진돗개와 세 마리의 새끼가 서로 엉겨 있는 모양새와 호기심 넘치는 표정에서 따뜻한 가족애가 느껴집니다. 작가는 세밀한 관찰을 통해 어미개의 뻣뻣한 털과 새끼의 보송보송한 감촉을 화면에 옮겨냈습니다. 노랗게 핀 개나리꽃과 푸른 풀밭 배경은 싱그러운 봄날의 한가로운 한 때를 더욱 부각시키고 있습니다. 이 작품은 동아시아 문화권에서 영모화의 오랜 주제였던 모견도를 현대 동양화로 재해석한 것입니다.
# author_description:  운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장애를 갖게 되었습니다. 17세에 이당(以堂) 

### 중복 처리

In [179]:
du_man_idx = df[df.duplicated(subset='관리번호', keep='last')]['관리번호'].values
df[df['관리번호'].isin(du_man_idx)]

,번호,작가,작품명,제작연도,재료,규격,부문,관리번호,작품 설명,작품이미지,이미지,art_description,author_description
90,91,장욱진 CHANG Ucchin,나룻배,1951,패널에 유화 물감,13.7×29,회화 II,10200.0,"장욱진(張旭鎭, 1917-1990)은 충남 연기군에서 출생하였으며, 경성 제2고등보...",1,91.jpg,"장욱진(張旭鎭, 1917-1990)은 충남 연기군에서 출생하였으며, 경성 제2고등보...","장욱진(張旭鎭, 1917-1990)은 충남 연기군에서 출생하였으며, 경성 제2고등보..."
95,96,장욱진 CHANG Ucchin,소녀,1939,캔버스에 유화 물감,29×13.7,회화 II,10200.0,"장욱진(張旭鎭, 1917-1990)은 충남 연기군에서 출생하였으며, 경성 제2고등보...",1,96.jpg,"장욱진(張旭鎭, 1917-1990)은 충남 연기군에서 출생하였으며, 경성 제2고등보...","장욱진(張旭鎭, 1917-1990)은 충남 연기군에서 출생하였으며, 경성 제2고등보..."


In [ ]:
du_idx = [33, 39, 40, 127, 129, 132]

In [ ]:
df = df[~df.index.isin(du_idx)].reset_index(drop=True)

## 데이터 저장

In [224]:
df.to_csv("../data/arts_all02.csv", index=False)

In [225]:
df = pd.read_csv("../data/arts_all02.csv")